In [ ]:
import igl
import numpy as np
from scipy.io import savemat
import os
from PyRMT import RMTMesh
import sys


sys.path.append('../')
from visual_utils import *
from icp_utils import *

#from evaluation import *

def visu(vertices):
    
    "The function normalizes the values over the vertices"
    
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap



In [ ]:
from scipy.io import loadmat
a=[]
PATH_bm = r"D:\Make_Galileo_great_again\data\giorgio_\MNI_remesh.mat"
PATH2 = " D:\Make_Galileo_great_again\data\giorgio_\bm10.mat "
a.append(loadmat(PATH_bm))


In [ ]:
from pyFM.mesh import TriMesh
target_re = TriMesh(r'D:\Make_Galileo_great_again\data\giorgio_\MNI_remesh.off')

In [ ]:
target_re.vertlist.shape

In [ ]:
SamplesDir = r"D:\Make_Galileo_great_again\data\giorgio_"

N_meshes = 0
m_list=[]
m_rem_list=[]
bm_list=[]

for i in range(N_meshes):
    v, f = igl.read_triangle_mesh(os.path.join(SamplesDir, fr"0000{i:02d}_tumoredbrain\0000{i:02d}_tumoredbrain.off"))
    v = np.asfortranarray(v)
    f = np.asfortranarray(f)

    m = RMTMesh(v, f)
    m.make_manifold()
    m_list.append(m)

    m_rem = m.remesh(6000)
    m_rem.clean_up()
    m_rem_list.append(m_rem)

    bm = m_rem.baryc_map(v)
    bm_list.append(bm)
    
    igl.write_triangle_mesh(os.path.join(SamplesDir, fr"00000{i:02d}_tumoredbrain.off"), m_rem.vertices, m_rem.triangles)

v, f = igl.read_triangle_mesh(r"D:\Make_Galileo_great_again\data\giorgio_\MNI.off")
v = np.asfortranarray(v)
f = np.asfortranarray(f)

m = RMTMesh(v, f)
m.make_manifold()
m_rem = m.remesh(6000)
m_rem.clean_up()
bm = m_rem.baryc_map(v)
igl.write_triangle_mesh(os.path.join(SamplesDir, "MNI_remesh.off"), m_rem.vertices, m_rem.triangles)
savemat(os.path.join(SamplesDir, "MNI_remesh.mat"), {"U" : bm})

In [ ]:
from pyFM.mesh import TriMesh

m0= TriMesh(m_list[0].vertices, m_list[0].triangles)
m0_rem=TriMesh(m_rem_list[0].vertices, m_rem_list[0].triangles)
m1 = TriMesh(m_list[1].vertices, m_list[1].triangles)
m1_rem = TriMesh(m_rem_list[1].vertices, m_rem_list[1].triangles)
m2 = TriMesh(m_list[2].vertices, m_list[2].triangles)
m2_rem = TriMesh(m_rem_list[2].vertices, m_rem_list[2].triangles)

In [ ]:
p2p_KNN = knn_query_normals(m0.vertlist, m0_rem.vertlist,
                                            m0.vertex_normals, m0_rem.vertex_normals,
                                            k_base=20, n_jobs=20, verbose=False)
#p2p_KNN = knn_query(meshlist[0].vertlist, meshlist[1].vertlist)

In [ ]:
plot_mesh(m0, color=np.mean(m0.vertices,axis=1))
plot_mesh(m0_rem, color=np.mean(m0.vertices,axis=1)[p2p_KNN])

KNN

In [ ]:
p2p_KNN = knn_query_normals(m0.vertlist, m0_rem.vertlist,
                                            m0.vertex_normals, m0_rem.vertex_normals,
                                            k_base=20, n_jobs=20, verbose=False)
#p2p_KNN = knn_query(meshlist[0].vertlist, meshlist[1].vertlist)

In [ ]:

p2p_KNN = knn_query(m0_rem.vertlist, m1_rem.vertlist)

In [ ]:
plot_mesh(m0_rem, color=np.mean(m0_rem.vertices,axis=1))
plot_mesh(m1_rem, color=np.mean(m0_rem.vertices,axis=1)[p2p_KNN])

In [ ]:
#p2p_KNN = knn_query_normals(m0_rem.vertlist, m2_rem.vertlist,
#                                            m0_rem.vertex_normals, m2_rem.vertex_normals,
#                                            k_base=20, n_jobs=20, verbose=False)
p2p_KNN = knn_query(m0_rem.vertlist, m2_rem.vertlist)

In [ ]:
plot_mesh(m0_rem, color=np.mean(m0_rem.vertices,axis=1))
plot_mesh(m2_rem, color=np.mean(m0_rem.vertices,axis=1)[p2p_KNN])

ZOOMOUT

In [ ]:
import pyFM.spectral as spectral
from pyFM.FMN import FMN
import pyFM

In [ ]:
from pyFM.refine.zoomout import mesh_zoomout_refine_p2p

Calculate the Laplacian basis

In [ ]:
m0_rem.process(k=150,intrinsic=True)
m2_rem.process(k=150,intrinsic=True)

Apply ZoomOut

In [ ]:
# apply zoomout starting from the Knn initialization
FM_12_zo, p2p_ZO = mesh_zoomout_refine_p2p(p2p_21=p2p_KNN, mesh1=m0_rem, mesh2=m2_rem, k_init=20, nit=26, step=5, return_p2p=True, n_jobs=10, verbose=True)

plot_mesh(m0_rem,color=np.mean(m0_rem.vertices,axis=1))
plot_mesh(m2_rem,color=np.mean(m0_rem.vertices,axis=1)[p2p_ZO])

Import original meshes

In [ ]:
#1 min e 11 sec.
import trimesh
#Number of meshes to import:
N_meshes= 3
meshlist=[]
for i in range(N_meshes):
    meshlist.append(TriMesh(fr'D:\Make_Galileo_great_again\data\giorgio_\0000{i:02d}_tumoredbrain\0000{i:02d}_tumoredbrain.off')) 
    mesh= trimesh.Trimesh(meshlist[i].vertices, meshlist[i].faces)
    components = mesh.split(only_watertight=False)
    largest_component = max(components, key=lambda comp: comp.volume) 
    print(f"Number of vertices before LC for mesh{i:02d}:", meshlist[i].vertices.shape)
    #meshlist[i]= TriMesh(largest_component.vertices, largest_component.faces).process(k=200, intrinsic=True)
    #print(f"Number of vertices after LC for mesh{i:02d}:", meshlist[i].vertices.shape)


Correspondence with upscaled FMap + Transferring function


In [ ]:
p2p_RM = spectral.FM_to_p2p(FM_12_zo, bm_list[0] @ m0_rem.eigenvectors, bm_list[2] @ m2_rem.eigenvectors, n_jobs=10)

Plot

In [ ]:
plot_mesh(meshlist[0],color=np.mean(meshlist[0].vertices,axis=1))
plot_mesh(meshlist[2],color=np.mean(meshlist[0].vertices,axis=1)[p2p_RM])

## Rematching with FunctionalMaps

### ICP

In [ ]:
subsample_list = np.zeros((3, 10), dtype=int)
for i in tqdm(range(3)):
    subsample_list[i] = meshlist[i].extract_fps(10, geodesic=False, verbose=False)
fps1 = subsample_list[0]
fps2 = subsample_list[2]

In [ ]:
# Get initial correspondences with KNN
p2p_21_init_sub = knn_query_normals(meshlist[0].vertlist[fps1], meshlist[2].vertlist[fps2],
                                            meshlist[0].vertex_normals[fps1], meshlist[2].vertex_normals[fps2],
                                            k_base=5, n_jobs=20, verbose=False)
# ICP Align the shape
_, R, t = icp_align(meshlist[2].vertlist[fps2], meshlist[0].vertlist[fps1],
                            p2p_12=p2p_21_init_sub,
                            return_params=True, n_jobs=20, epsilon=1e-4, verbose=False)

meshlist[2].rotate(R)
meshlist[2].translate(t)


### Rematching

In [ ]:
rem_list_tri=[m0_rem, m1_rem, m2_rem]

SRC=0
TRG=2


In [ ]:
subsample_list = np.zeros((3, 10), dtype=int)
for i in tqdm(range(3)):
    subsample_list[i] = rem_list_tri[i].extract_fps(10, geodesic=False, verbose=False)

KNN

In [ ]:
fps2 = knn_query(rem_list_tri[TRG].vertlist, rem_list_tri[SRC].vertlist[subsample_list[SRC]])

landmarks=np.array([subsample_list[SRC],fps2]).T

### Functional Maps

In [ ]:
from pyFM.functional import FunctionalMapping

process_params = {
        'n_ev': (20,20),  # Number of eigenvalues on source and Target
        'landmarks': landmarks,
        'subsample_step': 1,  # In order not to use too many descriptors
        'n_descr': 40, #number of descriptors
        'descr_type': 'HKS',  # WKS or HKS
    }
model = FunctionalMapping(rem_list_tri[SRC],rem_list_tri[TRG])
model.preprocess(**process_params,verbose=False)

model.fit(w_descr= 1e-1, w_lap= 1e-3, w_dcomm= 1,w_orient= 0, verbose=False)
fmap12=model.FM       #C{XY}, or A_{YX}^T

#p2p=model.get_p2p(fmap12, model.mesh1.eigenvectors[:,:k],model.mesh2.eigenvectors[:,:k],adj, bijective)
p2p=model.get_p2p()


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(fmap12)

In [ ]:
#cmap1 = np.mean(visu(rem_list_tri[SRC].vertlist),axis=1)
#cmap2_wks = cmap1[p2p]

plot_mesh(rem_list_tri[SRC],color=np.mean(rem_list_tri[SRC].vertices,axis=1))
plot_mesh(rem_list_tri[TRG],color=np.mean(rem_list_tri[SRC].vertices,axis=1)[p2p])

In [ ]:
#zoomout
from pyFM.refine.zoomout import mesh_zoomout_refine_p2p
FM_12_hks_zo, p2p_21_hks_zo = mesh_zoomout_refine_p2p(p2p_21=p2p, mesh1=rem_list_tri[SRC], mesh2=rem_list_tri[TRG], k_init=20, nit=16, step=5, return_p2p=True, n_jobs=10, verbose=True)

#cmap1 = np.mean(visu(rem_list_tri[SRC].vertlist),axis=1)
#cmap2_wks = cmap1[p2p_21_wks_zo]


plot_mesh(rem_list_tri[SRC],color=np.mean(rem_list_tri[SRC].vertices,axis=1))
plot_mesh(rem_list_tri[TRG],color=np.mean(rem_list_tri[SRC].vertices,axis=1)[p2p_21_hks_zo])

In [ ]:
plt.imshow(FM_12_hks_zo)

In [ ]:
p2p_ZO_RM = spectral.FM_to_p2p(FM_12_hks_zo, bm_list[SRC] @ rem_list_tri[SRC].eigenvectors, bm_list[TRG] @ rem_list_tri[TRG].eigenvectors, n_jobs=10)

In [ ]:
plot_mesh(meshlist[SRC],color=np.mean(meshlist[SRC].vertices,axis=1))
plot_mesh(meshlist[TRG],color=np.mean(meshlist[SRC].vertices,axis=1)[p2p_ZO_RM])